In [ ]:
#!/usr/bin/env python3
import os, sys
import numpy as np
import qutip as qt
import itertools

import scipy.linalg
import scipy.sparse
from scipy.integrate import solve_ivp

import multiprocessing as mp

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D

from correlator_methods import mat_zxy_to_pzm, vec_zxy_to_pzm, op_ln_val_ZXY, \
    get_deriv_op_vec, sandwich_deriv_op_vec, add_deriv_op_vecs, multiply_deriv_op_vecs, \
    deriv_op_vec_to_vals, deriv_vals_to_correlators

from time import time

np.set_printoptions(linewidth = 200)

In [ ]:
vecs_A, vecs_B = {}, {}
vecs_At_B, vecs_B_At = {}, {}
vecs_OTOC = {}
corrs_TTC, corrs_TTC_exact = {}, {}
corrs_OTOC, corrs_OTOC_exact = {}, {}
all_dicts = ( vecs_A, vecs_B, vecs_At_B, vecs_B_At, vecs_OTOC,
              corrs_TTC, corrs_TTC_exact, corrs_OTOC, corrs_OTOC_exact )

vecs_A_str = "vecs_A"
vecs_B_str = "vecs_B"
vecs_At_B_str = "vecs_At_B"
vecs_B_At_str = "vecs_B_At"
vecs_OTOC_str = "vecs_OTOC"
dec_0 = "dec_0"
dec_M = "dec_M"
dec_m = "dec_m"
dec_P = "dec_P"
dec_p = "dec_p"
dec_Z = "dec_Z"
dec_z = "dec_z"
NAT = "NAT"
OAT = "OAT"
TAT = "TAT"
TNT = "TNT"
TNB = "TNB"
all_methods = [ NAT, OAT, TAT, TNT, TNB ]

save_dir = "../data/multi_time/"
fig_dir = "../figures/multi_time/"
tmp_dir = "./tmp/"

In [ ]:
order_cap = 20 # order limit for short-time correlator expansion
N = 10**4 # number of spins
methods = [ OAT, TAT, TNT ]

def save_file_name(base_name, method, dec):
    return save_dir + f"{base_name}_{method}_{dec}_N{N}_O{order_cap}.txt"

time_steps = 200
max_time = 4 # maximum timulation time in units with N \chi = 1
times = np.linspace(0, max_time / N, time_steps)

ivp_tolerance = 1e-13 # relative error tolerance in numerical integrator

S = N/2
h_zxy = {}
h_zxy[NAT] = {}
h_zxy[OAT] = { (2,0,0) : 1 }
h_zxy[TAT] = { (2,0,0) : +1/3,
               (0,0,2) : -1/3 }
h_zxy[TNT] = { (2,0,0) : 1,
               (0,1,0) : S }
h_zxy[TNB] = { (2,0,0) : 1,
                (0,1,0) : -S }

init_state_X = (0,1,0)
init_state_nZ = (-1,0,0)
mat_X_to_nZ_zxy = -np.array([ [0,1,0], [1,0,0], [0,0,1] ])
mat_X_to_nZ_pzm = mat_zxy_to_pzm(mat_X_to_nZ_zxy)

collective_dec = 1/3
single_dec = collective_dec * S
dec_rates = { dec_0 : None,
              dec_p : ( (single_dec,0,0), (0,0,0) ),
              dec_z : ( (0,single_dec,0), (0,0,0) ),
              dec_m : ( (0,0,single_dec), (0,0,0) ),
              dec_P : ( (0,0,0), (collective_dec,0,0) ),
              dec_Z : ( (0,0,0), (0,collective_dec,0) ),
              dec_M : ( (0,0,0), (0,0,collective_dec) ) }
for single_dict in all_dicts:
    for dec in dec_rates:
        if single_dict.get(dec) is None:
            single_dict[dec] = {}

titles = { dec_0 : "No decoherence",
           dec_M : r"$\Gamma_-=\chi/3$",
           dec_m : r"$\gamma_-=S\chi/3$",
           dec_P : r"$\Gamma_+=\chi/3$",
           dec_p : r"$\gamma_+=S\chi/3$",
           dec_Z : r"$\Gamma_{\mathrm{z}}=\chi/3$",
           dec_z : r"$\gamma_{\mathrm{z}}=S\chi/3$" }

A_zxy = { (0,1,0) : 1,
          (0,0,1) : 1j }
B_zxy = { (0,1,0) : 1,
          (0,0,1) : -1j }

h_pzm = { method : vec_zxy_to_pzm(h_zxy[method]) for method in methods }
A_pzm = { method : vec_zxy_to_pzm(A_zxy) for method in methods }
B_pzm = { method : vec_zxy_to_pzm(B_zxy) for method in methods }

B_init_val = 0
for op, val in B_pzm[methods[0]].items():
    ln_val, phase = op_ln_val_ZXY(op, N)
    B_init_val += np.exp(ln_val) * phase

In [ ]:
##########################################################################################
# exact few-qubit simulation methods
##########################################################################################
exact_simulation_cutoff_size = 6

def comm(A, B): return A @ B - B @ A
def acomm(A, B): return A @ B + B @ A
def flat_time_deriv(flat_op_mat, hamiltonian, dec_sets):
    op_mat = flat_op_mat.reshape((2**N,2**N))
    time_deriv = 1j * comm(hamiltonian, op_mat)
    for dec_set in dec_sets:
        time_deriv += sum( dec_op[0].conj().T @ op_mat @ dec_op[0]
                           - 1/2 * acomm(dec_op[1], op_mat)
                           for dec_op in dec_set )
    return time_deriv.flatten()

def op_at_time(op_0, time, hamiltonian, dec_sets):
    if time == 0: return op_0
    time_deriv = lambda time, flat_op : flat_time_deriv(flat_op, hamiltonian, dec_sets)
    flat_op_t = solve_ivp(time_deriv, (0,time), op_0.flatten(), t_eval = [time],
                          rtol = ivp_tolerance, atol = ivp_tolerance).y[:,0]
    return flat_op_t.reshape((2**N,2**N))

if N <= exact_simulation_cutoff_size:

    I2 = qt.qeye(2)
    up = qt.basis(2,0)
    dn = qt.basis(2,1)
    II = qt.tensor([I2]*N)
    ZZ = 0 * II

    init_state_X_vec = qt.tensor([ up + dn ]*N).data.toarray() / 2**(N/2)

    # individual spin operators
    s_z = []
    s_p = []
    s_m = []
    for jj in range(N):
        s_z.append(qt.tensor([ I2 ] * jj + [ qt.sigmaz()/2 ] + [ I2 ] * (N-jj-1)))
        s_p.append(qt.tensor([ I2 ] * jj + [ qt.sigmap() ] + [ I2 ] * (N-jj-1)))
        s_m.append(qt.tensor([ I2 ] * jj + [ qt.sigmam() ] + [ I2 ] * (N-jj-1)))

    # collective spin operators
    S_p = sum(s_p)
    S_m = sum(s_m)
    S_z = sum(s_z)
    S_x =   1/2 * ( S_p + S_m )
    S_y = -1j/2 * ( S_p - S_m )

    # convert vector of operators (in zxy format) to a matrix
    def op_vec_to_mat(vec):
        mat = ZZ
        for op, val in vec.items():
            mat += val * S_z**op[0] * S_x**op[1] * S_y**op[2]
        return mat.data.toarray()

    # hamiltonian
    H_mat = { method : op_vec_to_mat(h_zxy[method]) for method in methods }
    A_mat = op_vec_to_mat(A_zxy)
    B_mat = op_vec_to_mat(B_zxy)

    # decoherence vectors
    dec_sets = {}
    for name, dec in dec_rates.items():
        dec_sets[name] = []
        if name is dec_0: continue
        for rate, ops in [ ( dec[0][0], s_p ), ( dec[0][1], s_z ), ( dec[0][2], s_m ),
                           ( dec[1][0], [S_p] ), ( dec[1][1], [S_z] ), ( dec[1][2], [S_m] ) ]:
            if rate == 0: continue
            dec_sets[name].append([ ( np.sqrt(rate) * op.data.toarray(),
                                      rate * (op.dag()*op).data.toarray() )
                                    for op in ops ])

In [ ]:
def expectation_value(op_mat):
    return ( init_state_X_vec.conj().T @ op_mat @ init_state_X_vec )[0,0]

if N <= exact_simulation_cutoff_size:

    start = time()

    for method, dec in itertools.product(methods, dec_rates):
        print(method, dec)
        At_mats = [ op_at_time(A_mat, time, H_mat[method], dec_sets[dec])
                    for time in times ]
        At_B_mats = [ At_mat @ B_mat for At_mat in At_mats ]
        B_At_mats = [ B_mat @ At_mat for At_mat in At_mats ]
        TTC_mats = [ At_B_mat - At_mat * B_init_val
                     for At_B_mat, At_mat in zip(At_B_mats, At_mats) ]
        comm_mats = [ At_B_mat - B_At_mat for At_B_mat, B_At_mat in zip(At_B_mats, B_At_mats) ]
        OTOC_mats = [ comm_mat.conj().T @ comm_mat for comm_mat in comm_mats ]

        corrs_TTC_exact[dec][method] \
            = np.array([ expectation_value(TTC_mat) for TTC_mat in TTC_mats ])
        corrs_OTOC_exact[dec][method] \
            = np.array([ expectation_value(OTOC_mat) for OTOC_mat in OTOC_mats ])
        corrs_OTOC_exact[dec][method] = np.real(corrs_OTOC_exact[dec][method])

    print(time()-start)

In [ ]:
##########################################################################################
# full simulation methods
##########################################################################################

init_state = { method : init_state_X for method in methods }
dec_mat = { method : None for method in methods }

for method in methods:
    if N < 10**4 or method in [ NAT, OAT ]: continue
    init_state[method] = init_state_nZ
    dec_mat[method] = mat_X_to_nZ_pzm
    h_pzm[method] = vec_zxy_to_pzm(h_zxy[method], init_state_X)
    A_pzm[method] = vec_zxy_to_pzm(A_zxy, init_state_X)
    B_pzm[method] = vec_zxy_to_pzm(B_zxy, init_state_X)

def compute_deriv_op_vecs(op, method, dec):
    return get_deriv_op_vec(order_cap, N, init_state[method], h_pzm[method],
                            dec_rates = dec_rates[dec], dec_mat = dec_mat[method],
                            deriv_ops = list(op.keys()), remove_irrelevant_ops = False)

def save_vec(vec, save_file):
    with open(save_file, "w") as f:
        for key in vec.keys():
            f.write(f"key {key}\n")
            for op, val in vec[key].items():
                f.write(f"{op} {val}\n")

def load_vec(load_file):
    vec = {}
    with open(load_file, "r") as f:
        for line in f:
            line_items = line.split()
            if line_items[0] == "key":
                key = eval(" ".join(line_items[1:]))
                vec[key] = {}
            else:
                op = eval(" ".join(line_items[:-1]))
                val = eval(line_items[-1])
                vec[key][op] = val
    return vec

def get_vec(base_name, method, dec, print_text = None):
    if print_text is not None: print(print_text)

    if base_name == vecs_A_str:
        if os.path.isfile(save_file_name(vecs_A_str, method, dec)):
            return load_vec(save_file_name(vecs_A_str, method, dec))
        else:
            vec = compute_deriv_op_vecs(A_pzm[method], method, dec)
            save_vec(vec, save_file_name(vecs_A_str, method, dec))
            return vec

    elif base_name == vecs_At_B_str:
        if os.path.isfile(save_file_name(vecs_At_B_str, method, dec)):
            return load_vec(save_file_name(vecs_At_B_str, method, dec))
        else:
            vec = sandwich_deriv_op_vec(vecs_A[dec][method], append_op = B_pzm[method])
            save_vec(vec, save_file_name(vecs_At_B_str, method, dec))
            return vec

    elif base_name == vecs_B_At_str:
        if os.path.isfile(save_file_name(vecs_B_At_str, method, dec)):
            return load_vec(save_file_name(vecs_B_At_str, method, dec))
        else:
            vec = sandwich_deriv_op_vec(vecs_A[dec][method], prepend_op = B_pzm[method])
            save_vec(vec, save_file_name(vecs_B_At_str, method, dec))
            return vec

    elif base_name == vecs_OTOC_str:
        if os.path.isfile(save_file_name(vecs_OTOC_str, method, dec)):
            return load_vec(save_file_name(vecs_OTOC_str, method, dec))
        else:
            vecs_At_B_comm = add_deriv_op_vecs(vecs_At_B[dec][method],
                                               vecs_B_At[dec][method], factor_rht = -1)
            vec = multiply_deriv_op_vecs(vecs_At_B_comm, vecs_At_B_comm, dag_lft = True)
            save_vec(vec, save_file_name(vecs_OTOC_str, method, dec))
            return vec

    else:
        print("basename for vector not recognized:",base_name)
        return None

In [ ]:
start = time()

for method, dec in itertools.product(methods, dec_rates):
    print(method, dec)

    vecs_A[dec][method] = get_vec(vecs_A_str, method, dec)
    vecs_At_B[dec][method] = get_vec(vecs_At_B_str, method, dec)
    vecs_B_At[dec][method] = get_vec(vecs_B_At_str, method, dec)

    deriv_vals_A = deriv_op_vec_to_vals(vecs_A[dec][method], N, init_state[method])
    deriv_vals_At_B = deriv_op_vec_to_vals(vecs_At_B[dec][method], N, init_state[method])

    all_corrs_A = deriv_vals_to_correlators(deriv_vals_A, times)
    all_corrs_At_B = deriv_vals_to_correlators(deriv_vals_At_B, times)

    corrs_A = sum( A_pzm[method][key] * corrs for key, corrs in all_corrs_A.items() )
    corrs_At_B = sum( A_pzm[method][key] * corrs for key, corrs in all_corrs_At_B.items() )

    corrs_TTC[dec][method] = corrs_At_B - corrs_A * B_init_val

print(time()-start)

In [ ]:
OTOC_order_cap = 20
free_cores = 2

if order_cap <= OTOC_order_cap:

    start = time()

    pool = mp.Pool(processes = mp.cpu_count() - free_cores)
    results = {}
    for method, dec in itertools.product(methods, dec_rates):
        args = ( vecs_OTOC_str, method, dec, f"{method} {dec}" )
        results[method,dec] = pool.apply_async(get_vec, args = args)
    pool.close()
    pool.join()

    for method, dec in results.keys():
        vecs_OTOC[dec][method] = results[method,dec].get()
        deriv_vals_OTOC = deriv_op_vec_to_vals(vecs_OTOC[dec][method], N, init_state[method])
        all_corrs_OTOC = deriv_vals_to_correlators(deriv_vals_OTOC, times)
        corrs_OTOC[dec][method] \
            = sum( np.real( A_pzm[method][key[0]] * np.conj(A_pzm[method][key[1]]) * corrs )
                   for key, corrs in all_corrs_OTOC.items() )

    print(time()-start)

In [ ]:
max_plot_time = 3.5 if N > exact_simulation_cutoff_size else 2
plot_methods = [ OAT, TAT, TNT ]
fontsize = 9

figsize_complex = (7,4.5)
legend_y_complex = -2.2
mag_phs_ratio = 2

figsize_real = (7,3.5)
legend_y_real = -0.85

dec_columns = len(dec_rates) // 2
dec_grid_map = { dec_0 : (1,0) }
for col, dec in enumerate([ dec_M, dec_P, dec_Z ]):
    dec_grid_map[dec] = (0,col)
for col, dec in enumerate([ dec_m, dec_p, dec_z ]):
    dec_grid_map[dec] = (1,col)

mag = "mag" # key for correlator magnitude
phs = "phs" # key for correlator phase

params = { "text.usetex" : True,
           "font.size" : fontsize,
           "axes.titlesize" : fontsize,
           "axes.labelsize" : fontsize,
           "legend.fontsize" : fontsize }
plt.rcParams.update(params)
plot_idx = N * times <= max_plot_time

plot_methods = [ method for method in plot_methods if method in methods ]
colors = [ "#4E79A7", "#F28E2B", "#E15759", "#76B7B2",
           "#59A14E", "#EDC949", "#B07AA2", "#FF9DA7",
           "#9C755F", "#BAB0AC" ]

def plot_complex(corrs, scale, filename, ylabels):
    dec_X_height_ratios = [ mag_phs_ratio, 1 ]
    dec_0_height_ratios = [ (1+mag_phs_ratio)/2, mag_phs_ratio, 1, (1+mag_phs_ratio)/2 ]

    axes = { dec : {} for dec in corrs.keys() }

    plt.figure(figsize = figsize_complex)
    full_grid = gridspec.GridSpec(1, 2, width_ratios = [ 1, dec_columns ])
    benchmarking_grid \
        = gridspec.GridSpecFromSubplotSpec(4, 1, height_ratios = dec_0_height_ratios,
                                           subplot_spec = full_grid[0])
    axes[dec_0][mag] = plt.subplot(benchmarking_grid[1])
    axes[dec_0][phs] = plt.subplot(benchmarking_grid[2])

    dec_grid = gridspec.GridSpecFromSubplotSpec(2, dec_columns, subplot_spec = full_grid[1])
    for dec in corrs.keys():
        if dec == dec_0: continue
        subgrid = gridspec.GridSpecFromSubplotSpec(2, 1, height_ratios = dec_X_height_ratios,
                                                   subplot_spec = dec_grid[dec_grid_map[dec]])
        axes[dec][mag] = plt.subplot(subgrid[0], sharex = axes[dec_0][mag],
                                     sharey = axes[dec_0][mag])
        axes[dec][phs] = plt.subplot(subgrid[1], sharex = axes[dec_0][mag],
                                     sharey = axes[dec_0][phs])

    for dec, corr in corrs.items():
        axis = axes[dec]
        axis[mag].set_title(titles[dec])

        for method, color in zip(plot_methods, colors):
            axis[mag].plot(N*times[plot_idx], abs(corr[method][plot_idx])/scale, color = color)
            axis[phs].plot(N*times[plot_idx], np.angle(corr[method][plot_idx])/np.pi,
                           color = color)

        plt.setp(axis[mag].get_xticklabels(), visible = False)
        if dec_grid_map[dec][0] == 1:
            axis[phs].set_xlabel(r"Time ($N\chi t$)")
        else:
            plt.setp(axis[phs].get_xticklabels(), visible = False)

        for val in [ mag, phs ]:
            if dec == dec_0:
                axis[val].set_ylabel(ylabels[val])
            else:
                plt.setp(axis[val].get_yticklabels(), visible = False)
                axis[val].yaxis.get_offset_text().set_visible(False)

            axis[val].tick_params(direction = "in")
            axis[val].grid(True)

    color_handles = [ Line2D([0], [0], color = color) for color in colors ]
    axes[dec_0][phs].legend(color_handles, plot_methods, loc = "lower center",
                            bbox_to_anchor = (0.5, legend_y_complex))

    plt.tight_layout()
    plt.savefig(filename)


def plot_real(corrs, scale, filename, ylabel):
    axes = { dec : {} for dec in corrs.keys() }

    plt.figure(figsize = figsize_real)
    full_grid = gridspec.GridSpec(1, 2, width_ratios = [ 1, dec_columns ])
    benchmarking_grid \
        = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec = full_grid[0])
    axes[dec_0] = plt.subplot(benchmarking_grid[1:3])

    dec_grid = gridspec.GridSpecFromSubplotSpec(2, dec_columns, subplot_spec = full_grid[1])
    for dec in corrs.keys():
        if dec == dec_0: continue
        axes[dec] = plt.subplot(dec_grid[dec_grid_map[dec]],
                                sharex = axes[dec_0], sharey = axes[dec_0])

    for dec, corr in corrs.items():
        axis = axes[dec]
        axis.set_title(titles[dec])

        for method, color in zip(plot_methods, colors):
            axis.plot(N*times[plot_idx], corr[method][plot_idx]/scale, color = color)

        if dec_grid_map[dec][0] == 1:
            axis.set_xlabel(r"Time ($N\chi t$)")
        else:
            plt.setp(axis.get_xticklabels(), visible = False)

        if dec == dec_0:
            axis.set_ylabel(ylabel)
        else:
            plt.setp(axis.get_yticklabels(), visible = False)
            axis.yaxis.get_offset_text().set_visible(False)

        axis.tick_params(direction = "in")
        axis.grid(True)

    color_handles = [ Line2D([0], [0], color = color) for color in colors ]
    axes[dec_0].legend(color_handles, plot_methods, loc = "lower center",
                       bbox_to_anchor = (0.5, legend_y_real))

    plt.tight_layout()
    plt.savefig(filename)

In [ ]:
%matplotlib agg
TTC_labels = { mag : r"$\left|C(t)\right|$", phs : r"$\phi(t)/\pi$" }
OTOC_label = r"$D(t)$"
if N <= exact_simulation_cutoff_size:
    plot_complex(corrs_TTC, S, tmp_dir + "two_time.pdf", TTC_labels)
    plot_complex(corrs_TTC_exact, S, tmp_dir + "two_time_exact.pdf", TTC_labels)
    plot_real(corrs_OTOC, S**2, tmp_dir + "OTOC.pdf", OTOC_label)
    plot_real(corrs_OTOC_exact, S**2, tmp_dir + "OTOC_exact.pdf", OTOC_label)
else:
    plot_complex(corrs_TTC, S, fig_dir + f"two_time_{order_cap}.pdf", TTC_labels)
    if order_cap <= OTOC_order_cap:
        plot_real(corrs_OTOC, S**2, fig_dir + f"OTOC_{order_cap}.pdf", OTOC_label)